In [3]:
import pandas as pd
glass= pd.read_csv('glass.csv')
glassx=glass.iloc[:,:-1]
glassy=glass.iloc[:,-1:]

In [4]:
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


kmeans = KMeans(n_clusters=10, init='k-means++', max_iter=500,random_state=0)
pipe = make_pipeline(StandardScaler(), kmeans).fit(glassx)
kmeans_cluster_labels = pipe.predict(glassx)

df = pd.DataFrame(data=glassx)
df['target'] = glassy
df['kmeans_cluster'] = kmeans_cluster_labels

result = df.groupby(['target'])['kmeans_cluster'].value_counts()
print(result)

target  kmeans_cluster
1       2                 35
        9                 23
        0                 12
2       2                 42
        0                 20
        8                  5
        9                  4
        3                  3
        4                  2
3       2                 12
        0                  3
        9                  2
5       3                  9
        6                  2
        0                  1
        7                  1
6       1                  3
        9                  3
        3                  2
        2                  1
7       1                 22
        7                  4
        9                  2
        5                  1
Name: kmeans_cluster, dtype: int64


In [5]:
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
import numpy as np

bandwidth = estimate_bandwidth(glassx)
print(bandwidth)
meanshift= MeanShift(bandwidth=bandwidth)
cluster_labels = meanshift.fit_predict(glassx)

print('cluster labels 유형:', np.unique(cluster_labels))

df['MeanShift'] = cluster_labels

result = df.groupby(['target'])['MeanShift'].value_counts()
print(result)

3.928368952309785
cluster labels 유형: [0 1 2 3 4 5 6 7 8]
target  MeanShift
1       0            47
        2            23
2       0            63
        4             5
        2             4
        1             2
        6             1
        7             1
3       0            15
        2             2
5       1            10
        5             2
        3             1
6       1             6
        3             3
7       1            22
        3             6
        8             1
Name: MeanShift, dtype: int64


In [6]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.6, min_samples=8, metric='euclidean')
dbscan_labels = dbscan.fit_predict(glassx)

print('dbscan labels 유형:', np.unique(dbscan_labels))

df['dbscan'] = dbscan_labels

result = df.groupby(['target'])['dbscan'].value_counts()
print(result)

dbscan labels 유형: [-1  0  1  2  3  4]
target  dbscan
1        0        32
        -1        21
         1         9
         2         8
2       -1        38
         3        38
3       -1        17
5       -1        13
6       -1         9
7       -1        20
         4         9
Name: dbscan, dtype: int64


In [7]:
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,target,kmeans_cluster,MeanShift,dbscan
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1,9,2,-1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1,2,0,0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1,2,0,0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1,2,0,0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7,1,1,-1
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7,1,1,4
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7,1,1,4
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7,1,1,4


In [8]:
from sklearn.preprocessing import scale

from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt



score_samples = silhouette_samples(glassx, df['kmeans_cluster'])
print('silhouette_samples( ) return 값의 shape' , score_samples.shape)
df['silhouette_coeff'] = score_samples
average_score = silhouette_score(glassx, df['kmeans_cluster'])
print('유리 데이터셋 kmeans Silhouette Analysis Score:{0:.3f}'.format(average_score))
print(df.head(3))
print('----------------------------------------------'*2)
score_samples2 = silhouette_samples(glassx, df['MeanShift'])
print('silhouette_samples( ) return 값의 shape' , score_samples2.shape)
df['silhouette_coeff2'] = score_samples2
average_score2 = silhouette_score(glassx, df['MeanShift'])
print('유리 데이터셋 MeanShift Silhouette Analysis Score:{0:.3f}'.format(average_score2))
print(df.head(3))
print('----------------------------------------------'*2)
score_samples3 = silhouette_samples(glassx, df['dbscan'])
print('silhouette_samples( ) return 값의 shape' , score_samples3.shape)
df['silhouette_coeff3'] = score_samples3
average_score3 = silhouette_score(glassx, df['dbscan'])
print('유리 데이터셋 dbscan Silhouette Analysis Score:{0:.3f}'.format(average_score3))
print(df.head(3))

silhouette_samples( ) return 값의 shape (214,)
유리 데이터셋 kmeans Silhouette Analysis Score:0.492
        RI     Na    Mg    Al     Si     K    Ca   Ba   Fe  target  \
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0       1   
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0       1   
2  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0       1   

   kmeans_cluster  MeanShift  dbscan  silhouette_coeff  
0               9          2      -1          0.689761  
1               2          0       0          0.439047  
2               2          0       0          0.469623  
--------------------------------------------------------------------------------------------
silhouette_samples( ) return 값의 shape (214,)
유리 데이터셋 MeanShift Silhouette Analysis Score:0.642
        RI     Na    Mg    Al     Si     K    Ca   Ba   Fe  target  \
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0       1   
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0       1   
2